# Libraries

In [ ]:
import os
import warnings
import xarray as xr
import rioxarray as rxr
from shapely.geometry import mapping
import geopandas as gpd
from rasterio.enums import Resampling
from datetime import datetime, timedelta
from calendar import monthrange

# Suppress PyGEOS warning for compatibility with Shapely 2.0
os.environ['USE_PYGEOS'] = '0'
warnings.filterwarnings('ignore', category=UserWarning, module='geopandas')

In [ ]:
# Set the main directory using an environment variable or fallback to a relative path
main_dir = os.getenv('PROJECT_DIR', os.path.join(os.getcwd(), 'data', 'ANIN', 'Generating Indices', 'FAPAR anomaly'))

# Directory for real-time data
Monthly_Copernicus_RT1 = os.path.join(main_dir, 'Copernicus', 'Monthly_RT1')

# Directories for historical mean/standard deviation data
Mean = os.path.join(main_dir, 'Mean')
STD = os.path.join(main_dir, 'std')

In [ ]:
# List of years folders for monthly data
Copernicus_years = sorted(os.listdir(Monthly_Copernicus_RT1))
# Dictionary includes sub-dictionaries for monthly data in each year
years_copernicus = {}

# Nested loop through each year and month within
for year in Copernicus_years:
    months_copernicus = {}
    for month in sorted(os.listdir(os.path.join(Monthly_Copernicus_RT1, year))):
        path = os.path.join(Monthly_Copernicus_RT1, year, month)
        image = rxr.open_rasterio(path, masked=True)
        months_copernicus[month[4:-4]] = image
    years_copernicus[year] = months_copernicus

In [ ]:
# Dictionaries of mean and std FAPAR
FAPAR_mean = {}
FAPAR_std = {}

# Lists of mean and std FAPAR
Mean_FAPAR = sorted(os.listdir(Mean))
STD_FAPAR = sorted(os.listdir(STD))

# Loop through each monthly mean FAPAR
for month in Mean_FAPAR:
    path = os.path.join(Mean, month)
    image = rxr.open_rasterio(path, masked=True)
    FAPAR_mean[month[:-9]] = image

# Loop through each monthly std FAPAR
for month in STD_FAPAR:
    path = os.path.join(STD, month)
    image = rxr.open_rasterio(path, masked=True)
    FAPAR_std[month[:-8]] = image

In [ ]:
for year in years_copernicus:
    for month in years_copernicus[year]:
        Monthly_FAPAR = years_copernicus[year][month]
        mean_FAPAR = FAPAR_mean[month]
        std_FAPAR = FAPAR_std[month]
        # Calculate FAPAR Anomaly
        FAPAR_Anomaly = (Monthly_FAPAR - mean_FAPAR) / std_FAPAR

        # Export FAPAR Anomaly as a TIFF file
        output_path = os.path.join(main_dir, 'FAPAR Anomaly', 'RT1', f'{month}{year}_FAPAR_Anomaly.tif')
        FAPAR_Anomaly.rio.to_raster(output_path)
print('FAPAR Anomaly files are created')